## Book 2: Adding Neighborhood Coordinates for Toronto, Canada - Combined Dataframe

Peer-graded assignment for IBM Data Science Professional certification capstone course: Week 3

**Tools used**:
- [BeautifulSoup](https://https://beautiful-soup-4.readthedocs.io/en/latest/)
- [Foursquare API tool](https://foursquare.com)
- [http://cocl.us/Geospatial_data](http://cocl.us/Geospatial_data)

Toronto neighborhoods information is loaded using postal codes pulled from Wikipedia using BeautifulSoup, then analyzed using the Foursquare API tool.

This project uses the following Wiki page: [List of postal codes of Canada: M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) 

### Steps to analyze Toronto Neighborhood Data

1. Load the wiki containing postal codes for Toronto beginning with the letter 'M' into a Pandas dataframe. BeautifulSoup is used to scrape the wiki for the postal data. 
2. The data is cleaned up for processing.
3. Load the Geospatial data from a .CSV file built from [http://cocl.us/Geospatial_data](http://cocl.us/Geospatial_data). This provides latitude and longitude parameters to use in Foursquare.
4. Build out the dataframe with the Toronto neighborhoods including the map coordinates.
5. Analyze and model the data to gain insights into neighborhood characteristics.

**This notebook includes the prior work for phase 1, and new steps for phase 2, adding latitude and longitude coordinates for neighborhoods based on their postal code.**


In [1]:
# Import Pandas and Beautiful Soup libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Link to source data and parsing of Wiki website using BeautifulSoup

In [2]:
# Use Beautiful Soup to get the postal code data from the wiki page
sourcelink = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(sourcelink).text
Postal_Info = BeautifulSoup(source, 'lxml')

### Set up Pandas dataframe to hold postal data

In [3]:
# Build dataframe with postal data
Column_Names = ['PostalCode','Borough','Neighborhood']
Postal_Data = pd.DataFrame(columns=Column_Names)

### Initialize variables for postal code, borough, and neighborhood, then loop through table rows to pull data

In [4]:
content = Postal_Info.find('div', class_='mw-parser-output')
postal_table = content.table.tbody
postal_code = 0
borough = 0
neighborhood = 0

for tr in postal_table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postal_code = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2:
            neighborhood = td.text.strip('\n').replace(']','')
            Postal_Data = Postal_Data.append({'Postalcode': postal_code,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [65]:
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Data cleaning - remove boroughs with 'Not assigned',  0 values

In [5]:
Postal_Data = Postal_Data[Postal_Data.Borough!='Not assigned']
Postal_Data = Postal_Data[Postal_Data.Borough!= 0]
Postal_Data.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,Postal_Data.shape[0]):
    if Postal_Data.iloc[i][2] == 'Not assigned':
        Postal_Data.iloc[i][2] = Postal_Data.iloc[i][1]
        i = i+1
df = Postal_Data.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [6]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]   

### Get first 5 and last 5 rows of data from the postal dataframe

In [7]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df.tail()

,Postalcode,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


### Group data by Postal Code and Borough

In [9]:
def neighborhood_list(grouped):
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [12]:
print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Load Geospatial data from .csv file built on data from http://cocl.us/Geospatial_data ###

In [15]:
df3 = pd.read_csv('C:/users/brian/coordinates.csv', delimiter="|")
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Join two dataframes into combined set including coordinates, then view first 5 rows ###

In [20]:
neighborhood_coords = [df2, df3]
neighborhood_df = pd.concat(neighborhood_coords, axis=1)

In [21]:
neighborhood_df.head()

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### Remove the duplicate 'Postal Code' column, and view head of combined dataframe ###

In [18]:
del neighborhood_df['Postal Code']

In [19]:
neighborhood_df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
